In [18]:
from ipycanvas import MultiCanvas,hold_canvas
from ipywidgets import Image
import math
import random
import IPython.display
import time
import copy
import datetime as dt
from IPython.core.display import HTML
with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# Kalah Rules

### Game structure:
<p>The playing field consists of 6 houses per side and additionally one large house per player at the edge, the so-called storage. At the beginning of the game, six seeds are placed in each house except the storage.</p>

<b style="background-color:yellow; color:blue">Comment:</b>
The wikipedia page on [Kalah](https://en.wikipedia.org/wiki/Kalah) states that initially the houses contain only 4 seeds, not six.

![title](images/Board.png)

### Gameplay:
<p>Alternately, a player selects a house on his side of the board. The player then places the seeds in the following houses in a counter-clockwise direction. If visited, a seed is placed in the own storage, the opponent's storage is left out. The player's turn ends if all seeds are placed.

There are some special rules, that apply if the last seed is placed in the player's storage or an empty player's house:
1. If the last seed is placed in the own storage, it is the player's turn again. 
2. If the last seed is placed in an empty house on the player's own side of the board, this seed and all the seeds in the opposite house are placed in the player's storage. In that case, it is the opponent's turn.</p>

<b style="background-color:darkred; color:lightblue">Error:</b>  This is actually not correct.   If the opposite house is empty, then the seed is not placed in the players storage.  Your implementation is therefore wrong w.r.t. ths rule.

### End of the game:
<p>The game ends when all the houses of one player have been emptied. The opposing player then places all remaining seed in his storage.</p>

### Object of the game:
<p>The winner is the player who has more seeds in his storage at the end of the game.</p>

Source: http://www.kalaha.de/kalaha.htm (9.11.2021)
<b style="background-color:yellow; color:blue">Comment:</b> Why do you cite a German website if this paper is written in English?
Furthermore, the description on that webpage is wrong.  A better reference is http://gambiter.com/mancala/Kalah.html.

# Kalah game definition

### Basic definition:
<p>We define the game G as a six-tuple as follows so that a computer is able to play Kalah.</p>
<br>
<center><em>G = &ltStates, s0, Players, nextStates, finished, utility></em></center>

The components have the following meanings:

1. **States** is a set that contains all possible states of the game Kalah. A state of the game is represented by a list containing two lists, representing the houses of the two players. The first six values of each list represent the number of seeds in the player's houses represented by the letters **{A, B, C, D, E, F}**. The seventh and last value stands for the number of seeds in the corresponding player's storage. The start state of the game is for example:

<center><em>[[6,6,6,6,6,6,0], [6,6,6,6,6,6,0]]</em></center>

2. **s0 ∈ States** is the start state.

<b style="background-color:yellow; color:blue">Comment:</b> Use $\in$ instead of **ε**.

3. **Players** is a list that contains the players. Kalah is a game for exactly two players, therefore this game's **Players** list only contains two elements.
4. **nextStates** is a function which calculates a set of states that are reachable by one move from Player p in the state s. To do so, the function receives a state **s ε States** and a player **p ε Players**. The formula is given as follows:

<center><em>nextStates: States x Players &rarr; 2<sup>States</sup></em></center>

5. **finished** is a function that takes a state **s $\varepsilon$ States** and checks if the game is finished, meaning that one of the players has emptied all their houses . The formula is: 
<br><br>
<center><em>finished: States &rarr; B</em></center>
<br>
<b style="background-color:yellow; color:blue">Comment:</b> Use $\mathbb{B}$ instead of <em>B</em>.
The function finished is used to compute a set TerminalStates that contains all states from a finished game. This set is defined as follows: 
<br><br>
<center><em>TerminalStates:= {s ε States | finished(s)}</em></center>
<br>
6. **utility** is a function that calculates the value of the game for a player. Therefore, the function takes a state **s ε TerminalStates** and a player **p ε Players**. The value that the function returns is an element from the set **{-1, 0, 1}**. The player p has lost the game when the function returns -1. If the function returns 1, the player has won the game and if the value is 0, the game ends in a draw. The formula for the function is:

<center><em>utility: TerminalStates x Players &rarr; {-1, 0. 1}</em></center>

source: https://github.com/karlstroetmann/Artificial-Intelligence/blob/master/Lecture-Notes/artificial-intelligence.pdf, S.87, Abruf am 06.02.2022

# Classes

The Kalah game is implemented with the use of several classes, containing base class <tt>Kalah_Game</tt>, a <tt>Board</tt> class for displaying the UI and a <tt>Player</tt> class as well as several classes inheriting from this <tt>Player</tt> class:

- <tt>Human</tt>
- <tt>Repeated_Player</tt>
- <tt>Random_AI</tt>

The following illustration shows a class diagram of all classes involved:

<img src="images/Kalah_Class_Diagram.png" alt="Kalah_Class_Diagram" width="600"/>

<b style="background-color:yellow; color:blue">Comment:</b> 
I could not find some of the methods mentioned in this image.  For example, there is no method `choose_trough`.  Neither is there a method `calculate_winner`.  Actually, I don't believe in the value of diagramms of this kind for projects of this size.

This image was created with [Creatly](https://creately.com/).

## Global Variables

Additionally to the classes, there is one global variable *gOrder* with the purpose to help converting the state of the game board, which is represented by a nested list, to letters. The letter representation is mainly for the UI, but also distributes to print and logging messages being easier to humanly comprehend.

In [19]:
gOrder = [['A','B','C','D','E','F','O'],
         ['a','b','c','d','e','f','o']]

## Global Game Functions

- *other_player(player_num)*
- *move(state, player_num, choice)*
- *next_states(state, player_num)*
- *finished(state)*
- *utility(state, player_num)*
- *value(state, player_num)*

TODO

### Function: other_player
The function *other_player(player_num)* returns the opponent's player number to a given player number. This means that it returns 1 for the input 0, and 0 for the input 1.

In [20]:
def other_player(player_num):
    return (player_num + 1) % 2

### Function: move

<b style="background-color:yellow; color:blue">Comment:</b> All parameters of *move* should be included and explained.

The function *move(choice)* receives a state, the number of a player and the house index they have chosen as a result of the <tt>Player</tt> method *choose_house*. It calculates the actions of the player's turn and returns the resulting new game state.  <b style="background-color:yellow; color:blue">Comment: This is false!</b>
Additionally, the function is used in *next_states* to calculate all possible following states from one state.

##### The calculations are implemented based on the following game rules:

1. The current player choses one of his house. The seeds from the chosen house are placed counterclockwise in the house of both players and in the store of the current player. The store of the opponent is left out. Then it is the turn of the opponent.


2. If the last seed is placed in the store of the current player, they get another move.


3. If the last seed is placed in an empty house of the current player the seed from this house and all seeds from the opposite house are place in the store of the current player. Then it's the opponent's turn.

<b style="background-color:yellow; color:blue">Comment:</b> 
- Do not use `C`-syntax below.
- The idea of of variable names like `c_house_player_num` is difficult to guess.  What does the `c` stand for?
- The code of this function is rather long.  Wouldn't it be clearer if you introduced auxiliary methods to structure the code?
- A function as involved as the function `move` should have some unit tests.
- You should specify what exactly is returned.
- `other_player(player_num)` is called twice.

In [21]:
def move(state, player_num, choice):
    new_state = copy.deepcopy(state)

    seeds = new_state[player_num][choice]
    new_state[player_num][choice] = 0

    c_house_player_num = player_num
    c_house_num = choice

    # Go through houses counterclockwise
    while (seeds > 0):
        c_house_num += 1

        # Skip opponent's store
        if(c_house_num == 6 and c_house_player_num != player_num):
            continue

        # Switch to houses of the other player
        if(c_house_num > 6):
            c_house_num = 0
            c_house_player_num = other_player(c_house_player_num)

        # Add seed to the currently visited house
        new_state[c_house_player_num][c_house_num] += 1
        seeds -= 1
    
    another_turn = False
    # Check for special rules after last seed was placed in own store or own empty house:
    if(c_house_player_num == player_num):

        # Rule: Another turn if last seed is placed in own store
        if(c_house_num == 6) and not finished(new_state):
            # Give current player another turn
            another_turn = True

        # Rule: Last seed is placed in empty house of current player
        elif(new_state[player_num][c_house_num] == 1):                
            # Collect all seeds to be rewarded and empty both houses
            receivedSeeds = new_state[player_num][c_house_num]
            receivedSeeds += new_state[other_player(player_num)][5 - c_house_num]
            new_state[other_player(player_num)][5 - c_house_num] = 0
            new_state[player_num][c_house_num] = 0
            # Award all the seeds to the current player's kalah
            new_state[player_num][6] += receivedSeeds
            # if self.display_mode != 0:
                # print(f'Player {self.players[player_num].name} gets a steal for {receivedSeeds} Seeds!')

    move_log = tuple([player_num, choice, copy.deepcopy(new_state)])
    
    return new_state, another_turn, move_log

### Function: finished
The private function *finished()* checks if one of the players has no seeds in their house left and is therefore unable to take another turn. If this is the case, the function returns True, otherwise it returns False. 

<b style="background-color:yellow; color:blue">Comment:</b> 
This implementation is not very efficient, because the decision can be taken as soon as the first non-empty house is discovered.

Furthermore, code like 
```
    if(not (sum0 == 0 or sum1 == 0)):
        return False
    return True
```
could be simplified as follows:
```
    return sum0 == 0 or sum1 == 0
```

In [22]:
def finished(state):
    sum0 = sum(state[0][:-1])
    sum1 = sum(state[1][:-1])

    if(not (sum0 == 0 or sum1 == 0)):
        return False
    return True

### Function: next_states
The function *nextStates()* is a function which calculates a set of states that are reachable by one move from Player p in the state s. To do so, the function receives a state **s ε States** and a player **p ε Players**. The formula is given as follows:

<center><em>nextStates: States x Players &rarr; 2<sup>States</sup></em></center>

<b style="background-color:yellow; color:blue">Comment:</b> 
* The function is called `next_states` instead of `nextStates` and it returns a list instead of a set.
* Furthermore, this list does not contain states but pairs.  It is not obvious how these pairs are too be interpreted.
* You should use the same names for the parameters in the code as in the comment, e.g. **state** instead of **s**.

In [23]:
def next_states(state, player_num):
    states = []
    for choice in range(6):
        # Check if choice is valid (at least one seed in house)
        if state[player_num][choice] == 0:
            continue   
        next_state, another_turn, _ = move(state, player_num, choice)

        # Check if player has another turn
        # If so, next_states is called recursively until the player has no other turn 
        if another_turn:
            for s,choices in next_states(next_state, player_num):
                states.append((s,[choice] + choices))
        else:
            states.append((next_state,[choice]))

    return states

<b style="background-color:yellow; color:blue">Comment:</b> 
Unit tests like the test below should use `assert`.

In [24]:
next_states([[0, 0, 0, 0, 0, 1, 43], [0, 2, 0, 1, 2, 1, 22]],0)

[([[0, 0, 0, 0, 0, 0, 44], [0, 2, 0, 1, 2, 1, 22]], [5])]

### Function: utility
The  function *utility(player_num)* receives the number of the current player and uses the current state to calculate the utility of the state for the player. The function compares the seeds from both stores and returns an Integer dependent on the result. If the current player has more seeds than the opponent the function returns 1. If they own less seeds than the opponent the function returns -1 and if it is a draw the number 0 is returned.     

In [25]:
def utility(state, player_num):
    playerStore = state[player_num][6]
    opponentStore = state[other_player(player_num)][6]
    if(playerStore > opponentStore):
        return 1
    elif(playerStore == opponentStore):
        return 0
    else:
        return -1

<b style="background-color:darkred; color:lightblue">Error:</b> 
This is not correct because you have to take account of the remaining stones that are left in the houses!
These have to be added to the stones in the store.

### Function: value


#### Auxiliary functions for the method value

The method *to_tuple* takes a list of the two lists that represent a game state. The function converts the list to a tuple of tuples and returns it.

The method *to_list* does that procedure vice versa.

In [26]:
def to_tuple(state_list):
    return tuple(tuple(s) for s in state_list)

In [27]:
to_tuple([[1, 8, 1, 0, 1, 2, 16], [1, 4, 0, 4, 15, 14, 5]])

((1, 8, 1, 0, 1, 2, 16), (1, 4, 0, 4, 15, 14, 5))

In [28]:
def to_list(state_tuple):
    return list(list(s) for s in state_tuple)

In [29]:
to_list([[1, 8, 1, 0, 1, 2, 16], [1, 4, 0, 4, 15, 14, 5]])

[[1, 8, 1, 0, 1, 2, 16], [1, 4, 0, 4, 15, 14, 5]]

#### Function: memoize

The function *memoize* computes a memoized version of a function f that is given as parameter. At first a dictionary named Cache is ceated that is used as a memory cache for the function *memoized*. At first *memoized* tries to retrieve the value of the given function f from the dictionary Cache and returns the value. If *memoize* can't retrieve a value the function f is called to compute the result and the result is stored in the Cache.   

In [30]:
Cache = {}

def memoize(f):
    global Cache

    def f_memoized(*args):
        args = (to_tuple(args[0]),args[1],args[2])
        if args in Cache:
            return Cache[args]
        
        result = f(to_list(args[0]),args[1],args[2])
        Cache[args] = result
        return result

    return f_memoized

The function *value* takes a *state* and a *player*. In addition, a *limit* is given to define the recursion depth, which limits the number of next states which are calculated and evaluated. Similar to the function *utility*, the function *value* returns a value from the set {-1, 0. +1}. The base case of the recursive function *value* is the case that the game is finished or if the limit has reached the value 0. In these cases, the *utility* function is called. 

In the other cases, the next states are computed for both players alternately until the limit is reached or the game reaches a finished state. Bacause the gains of the oppoenent are the losses of player p, the negative output of *value(n, o)* is taken when calculating the current game value to the opponent.   

In [31]:
@memoize
def value(state, player_num, limit):
    limit -= 1
    if finished(state) or next_states(state, player_num)==[] or limit==0:
        return utility(state, player_num)
    other = other_player(player_num)
    return max([-value(ns, other,limit) for ns,_ in next_states(state, player_num)])

<b style="background-color:yellow; color:blue">Comments:</b> 
* How is it possible that `finished(state)` is `False` but `next_states(state, player_num)==[]`?
* This is not good enough without the use of a heuristic.  If `limit == 0`, then instead of
  returning the `utility`, a value computed by a heuristic should be computed.
* You should analyze how much the program speeds up because of memoization.
* Logistische Funktion von Verhältnis - 1.

In [32]:
print(value([[1, 8, 1, 0, 1, 2, 16], [1, 4, 0, 4, 15, 14, 5]], 0, 3))

1


# Player Class

#### Attributes:
- *player_type*
- *number*
- *name*

#### Methods:

- *\_\_init__(number, name)*
- *\_\_str__()*
- *_available_house(own_house)*
- *choose_house(current_state)*

The class <tt>Player</tt> is the superclass which represents a Kalah game player. In Kalah there are exactly two players which compete against each other. 

The attribute *player_type* is only for subclasses of the <tt>Player</tt> class. It contains a string representation of the name of the <tt>Player</tt> subclass. This makes it possible to print the specific type of a <tt>Player</tt> object. This is for example used for creating log files. 

The attribute *number* of a player has either the value 0 or 1 depending on the order in which the players take their turns.
If the number of the player is 0, they start the game by having the first turn.
Additionally, *number* represents the index of the state list which contains the player's list of house values.

The attribute *name* is a string which represents the player's name in the UI or print and logging messages.

### Method: __init__
The method *\_\_init__(number, name)* initializes the <tt>Player</tt> object and checks if the given *number* is either 0 or 1 and if the given *name* is a string. If one of these criteria is not met, an error message is raised.

### Method: __str__
The method *\_\_str__()* defines the *name* of the <tt>Player</tt> object as their string representative for print messages.

In [33]:
class Player:
    
    def __init__(self, number, name):
        
        if number in [0,1]:
            self.number = number
        else:
            raise ValueError("Number of player must be 0 or 1!")
        if isinstance(name, str):
            self.name = name
        else:
            raise ValueError("Name must be a string!")
            
    def __str__(self):
        return self.name

### Method: _available_houses

The private method *_available_houses(own_houses)* receives a list which represents the player's houses and their store. The method returns a list with the indizes of all houses which contain at least one seed (list indizes<b style="background-color:darkred; color:lightgreen">indices</b> with a value higher than null).

In [34]:
def _available_houses(self, own_houses):
    available = []
    for i in range(len(own_houses)):
        if own_houses[i] != 0:
            available.append(i)
    return available

Player._available_houses = _available_houses

<b style="background-color:yellow; color:blue">Comments:</b>  
The body of the function `_available_houses` can be written in one line:

In [35]:
def _available_houses(self, own_houses):
    return [i for i, n in enumerate(own_houses) if n != 0]

Player._available_houses = _available_houses

### Method: choose_house

The method *choose_house(current_state)* receives the current state of the Kalah board and returns the index of the chosen house from the player's house list. <s>This method differentiates the <tt>Player</tt> subclasses</s> <b style="background-color:darkred; color:lightgreen">This method is used to distinguish the player subclasses</b>
and is therefore <s>differently implemented</s> <b style="background-color:darkred; color:lightgreen">implemented differently</b> in each of them. There is no basic implementation of this method in the <tt>Player</tt> class. This is why in the class <tt>Kalah_Game</tt> only subclasses of <tt>Player</tt> are accepted as players. Therefore, this class is intended as an abstract class.

In [36]:
def choose_house(self, current_state):
    pass

Player.choose_house = choose_house

## Tests of Player Class

### 1. Player number must be 0 or 1

In [37]:
try:
    Player(2, "Player1")
except ValueError as e:
    print(e)

Number of player must be 0 or 1!


### 2. Player name must be a string

In [38]:
try:
    Player(0, 3)
except ValueError as e:
    print(e)

Name must be a string!


### 3. Successful Player creation

In [39]:
Player(0, "Player1")

## Human Player Class

The class <tt>Human</tt> inherits from <tt>Player</tt>.
It is used for humans playing against each other or against one of the AIs. 

<b style="background-color:yellow; color:blue">Comments:</b>
The parameters of all methods should be discussed.

In [40]:
class Human(Player):
    
    def __init__(self, number, name):
        self.player_type = "Human"
        super().__init__(number, name)

### Method: choose_house
The method *choose_house(current_state)* for the <tt>Human</tt> class is implemented as follows:

At first the own <s>house</s><b style="background-color:darkred; color:lightgreen">houses</b> are extracted from the game state and the available house <s>indizes</s><b style="background-color:darkred; color:lightgreen">indices</b> are calculated using the method *_available_houses*. Afterwards the human player is asked to choose one of the available house via a input field. With limiting the player to the available house, it is easier to detect a correct input. If the player enters an <s>unvalid</s><b style="background-color:darkred; color:lightgreen">invalid</b> value, they are asked to input a value again until a valid value is given. The index of the corresponding house is returned.

In [41]:
def choose_house(self, current_state):
    own_t,store = current_state[self.number][:6],current_state[self.number][6]
    available = self._available_houses(own_t)

    i_string = "Choose one of the available houses:\n"
    for i in available:
        i_string += f"{gOrder[self.number][i]}, "

    choice_str = ""
    letter_numbers = {gOrder[1][i]:i for i in range(6)}
    
    while choice_str not in [k for k in letter_numbers if letter_numbers[k] in available]:
        choice_str = input(i_string[:-2]+"\n").lower()

    choice = letter_numbers[choice_str]
    IPython.display.clear_output()
    return choice
    
Human.choose_house = choose_house

### Test of Human Player Class

In [42]:
human = Human(1,"Hans")

In [43]:
#human.choose_house([[6,6,6,6,6,6,0], [6,0,6,6,4,6,0]])

## Repeated Player Class

The class <tt>Repeated_Player</tt> inherits from <tt>Player</tt>.
It is used for repeating the behavior of a player from a previous game on basis of a given log file.

For this reason it takes the **logged_moves** information from the logged game and saves it as *moves_to_repeat*, so that these moves can be replicated in the *choose_house* method.

In [44]:
class Repeated_Player(Player):
    
    def __init__(self, number, name, logged_moves):
        self.player_type = "Repeated_Player"
        self.moves_to_repeat = logged_moves
        super().__init__(number, name)

### Method: choose_house
The method *choose_house(current_state)* for the <tt>Repeated_Player</tt> class is implemented as follows:

The list *moves_to_repeat* is handled as a stack in this method. This means that one after one, the moves which stand at the beginning of the list are extracted and removed from the list. In the beginning of the method this process is repeated until the current state of the game matches the state of the last removed move from the *moves_to_repeat* list. This means, that the move which the player will do next is should now be at the beginning of the list.

In the next step, this move is also extracted from the list, but not removed as it could contain the current state for the next move of the player if they gain another turn. To make sure, that this move was actually done by this player in the logged game, the logged move player number is compared to the actual player number. If the numbers match, the choice from that move is returned.

In [45]:
def choose_house(self, current_state):
    
    state = self.moves_to_repeat.pop(0)[2]
    while state != current_state:
        if len(self.moves_to_repeat) == 0:
            raise ValueError("There is no move left to repeat!")
        state = self.moves_to_repeat.pop(0)[2]
        
    this_move = self.moves_to_repeat[0]
    if this_move[0] not in (-1,self.number):
        raise ValueError("This move was originally not played by this player!")
        
    choice = this_move[1]
        
    return choice

Repeated_Player.choose_house = choose_house

## Random_AI Player Class

The class <tt>Random_AI</tt> inherits from <tt>Player</tt>. It is a simple AI player implementation which chooses <s>house</s><b style="background-color:darkred; color:lightgreen">the houses</b> at random.

The class <tt>Random_AI</tt> has a modified *\_\_init__* function which has an additional argument *seed* for setting a seed for the **random** library functions. Whenever the same seed is set, the <tt>random</tt> function creates the same random numbers or chooses the same random objects from a list. Adding this argument eases the logging and debbuging of <tt>Random_AI</tt> players.

In [46]:
import random as rn

class Random_AI(Player):
    
    def __init__(self, number, name, seed):
        self.player_type = "Random_AI"
        self.seed = seed
        rn.seed(seed)
        super().__init__(number, name)

### Method: choose_house
The method *choose_house(current_state)* for the <tt>Random_AI</tt> class is implemented as follows:

At first the own <s>house</s> are extracted from the game state and the available house indizes are calculated using the method *_available_houses*. From the list of available house indizes, one is chosen at random using the function **choice** from the library **random**. Afterwards, this value is returned.

In [47]:
def choose_house(self, current_state):
    own_t,safe = current_state[self.number][:6],current_state[self.number][6]
    available = self._available_houses(own_t)
    choice = rn.choice(available)

    return choice

Random_AI.choose_house = choose_house

### Test of Random_AI Player Class

In [48]:
ai = Random_AI(0,"Rando", 1)

In [49]:
ai.choose_house([[6,6,6,6,6,6,0], [6,6,6,6,6,6,0]])

1

# Minimax Player Class

The class <tt>Minimax</tt> inherits from <tt>Player</tt>. It is an AI player implementation which chooses a following state by calculating all possible next states for a defined limit depth. It then chooses the option which guarantees them a higher number of seeds in their store than their opponent. If there are several options, the choice which results in the highest number of seeds in the player's store is chosen.

The class <tt>Minimax</tt> has a modified *\_\_init__* function which takes the additional argument *limit* which is only relevant to the method *choose_house*. There the value of *limit* defines, for how many recursion steps the function *value* should be called. This means that *limit* defines how many next game states should be calculated to form the decision of the <tt>Minimax</tt> AI.

The *limit* should be chosen carefully, as higher limits increase the recursion depth and computing time exponentially. Additionally, *limit* has to be greater than 1 at all times. 

<b style="background-color:yellow; color:blue">Comments:</b>
Only the computing time increases exponentially.

Ideally *limit* should be a number from 2 - 4 (incl.).

In [50]:
class Minimax(Player):
    
    def __init__(self, number, name, limit):
        self.player_type = "Minimax"
        self.limit = limit
        super().__init__(number, name)

### Method: choose_house
The method *choose_house(self,current_state)* is the core method of the <tt>Minimax</tt> algorithm. It calculates the best choices for a <tt>Minimax</tt> player, using the previously defined functions *value(state, player_num, limit)* and *next_states(state, player_num)*.

First, the best possible value from {-1, 0, 1} (meaning loss, draw or win) that can be reached with the current state of the game is calculated. Following that, all next possible states are checked and only those are elected as eligible, which reach the previously calculated best possible value.

Afterwards, the best choice is being made by comparing all eligible choices for the highest player's Kalah store value.

In [51]:
def choose_house(self, current_state):
    
    NS = next_states(current_state, self.number)
    bestVal = value(current_state, self.number,self.limit)
    
    BestChoices = [(choices[0],state[self.number][6]) for state,choices in NS if -value(state, other_player(self.number),self.limit-1) == bestVal]
    
    if len(BestChoices) == 0:
        raise ValueError(f'No choice for Minimax found! \nbestVal: {bestVal}\nBestChoices: {BestChoices}\nnext_states: {NS}')

    # Find best choice, where number of seeds in own Store is maximized (out of next_states)
    best_choice, best_store = BestChoices[0]
    for choice, store in BestChoices:
        if store > best_store:
            best_choice = choice
            best_store = store

    return best_choice

Minimax.choose_house = choose_house

<b style="background-color:yellow; color:blue">Comments:</b>
You should randomly choose one of the optimal moves.

In [52]:
Minimax(1,"Mini",3).choose_house([[7, 7, 6, 6, 6, 6, 0], [0, 0, 8, 8, 8, 8, 2]])

5

# Kalah_Game Class

<b style="background-color:yellow; color:blue">Comments:</b>
Kalah is usually played with only 4 seeds in every house.  There should be a parameter to control this number.

#### Attributes:
- *state*
- *players*
- *current_player*
- *display_mode*
- *logged_moves*

#### Methods:

- *\_\_init__(players)*
- *_show_state(state)*
- *show_state()*
- *draw_board()*
- *_draw_seeds()*
- *_draw_numbers()*
- *start()*
- *log_to_file(file_id)*

The class <tt>Kalah_Game</tt> is the core of the Kalah game implementation. It contains all information on the game, including the current game state.

The attribute *state* represents the state of the Kalah board which is defined by the number of seeds laying in each of the player's house and their stores. It is implemented by a nested list which contains a list for the house on each of the two players' sides. The last index of each of the lists is the store of that player. At the start of the game, where the stores are empty and there are six seeds in every house, the implemented representation of the state for example is: [[6,6,6,6,6,6,0], [6,6,6,6,6,6,0]].

The attribute *players* is a list of the two players that play the game. They must be instances of a subclass of the <tt>Player</tt> class. The order in which they take turns is determined by their <tt>Player</tt> *number*: The player with the number 0 goes first.

The attribute *current_player* has either the value 0 or 1. It takes track of which player's turn it currently is. If *current_player* has the value 0 for example, it is the turn of the player which stands at index 0 of the *players* list and therefore also has the <tt>Player</tt>'s class attribute *number* of value 0.

The attribute *display_mode* defines which way the game is displayed in the output console. The possible values are 0, 1 and 2 and are defined as follows:
- 0: No output is displayed. The game is only logged to the log file
- 1: The board is only displayed using the print method *_show_state*
- 2: The board is displayed with the function *draw_board* using **ipycanvas** for rendering

The attribute *logged_moves* is a list containing a tuple for every move that is made in the game. This list is necessary for the method *log_to_file*. Each tuple has the player number of the player that have made the move at the first index, the index of the chosen house at the second index and the resulting game state as the third index. The start state is saved as the first move in the list, having -1 as the player number and -1 as the chosen house number. As Python uses references, the states are saved in this list by creating deep copies with the library **copy**. At the beginning of the game *logged_moves* looks like this:

\[(-1, -1, [[6,6,6,6,6,6,0], [6,6,6,6,6,6,0]])\]

### Method: __init__
The method *\_\_init__(players)* initializes the game by setting the *state* to the initial state (seen above), initializing a board object and setting the *players* list using the received "players" argument. Before setting the *players* list, the received list is checked for the number of items it contains (must be exactly 2) and if the items in the list are both instances of a <tt>Player</tt> subclass (but not of the class <tt>Player</tt> itself). In error case, matching error messages are raised.

In [53]:
class Kalah_Game():
    
    def __init__(self, players, display_mode):
        self.state = [[4,4,4,4,4,4,0], [4,4,4,4,4,4,0]]
        self.turn = 0
        
        if len(players) != 2:
            raise ValueError("There must be exactly two players!")
        if not ((isinstance(players[0], Player) and type(players[0]) != Player) 
            and (isinstance(players[1], Player) and type(players[1]) != Player)):
            raise ValueError("Both players must be of instances of a subclass of the class Player!")
        if {players[0].number, players[1].number} != {0,1}:
            raise ValueError("One of the players must be self.number 0 and the other one self.number 1!")
        
        self.players = players
        self.current_player = 0
        
        if display_mode not in range(3):
            raise ValueError("The display mode must be 0, 1 or 2!")
        self.display_mode = display_mode
        
        self.logged_moves = [(-1,-1,copy.deepcopy(self.state))]

### Methods: _show_state and show_state
The private method *_show_state(state)* creates a formatted string which represents the received state and prints it to the console. It can be used as an alternative to the **ipycanvas** game UI.

The method *show_state()* calls the private method *_show_state(state)* with the current game state (attribute *state*).

In [54]:
def _show_state(self, state):
    s = f''

    s += f'{self.players[0].name}:\t\t\t'
    for j in range(6,-1,-1):
        s += f'{gOrder[0][j]}: {state[0][j]}  '
    s += f'\n'

    s += f'{self.players[1].name}:\t\t\t'
    for j in range(7):
        s += f'{gOrder[1][j]}: {state[1][j]}  '
    s += f'\n'

    print(s)

Kalah_Game._show_state = _show_state

In [55]:
def show_state(self):
    self._show_state(self.state)
    
Kalah_Game.show_state = show_state

### Method: draw_board
The method *draw_board()* sets the required positional values for the methods *_draw_seeds()* and *_draw_numbers()*.

It also translates the current state of the game to a dictionary so that the positional values can be related to the corresponding seed amount in the respective houses.

Additionally this function creates a MultiCanvas for the display of the board. The first layer is the background and the second layer is the foreground.

Afterwards this function calls the functions *_draw_seeds()* and *_draw_numbers()*.

Finally the finished canvas gets displayed.

In [56]:
def draw_board(self):
    TOP_ROW_POSITION = (175,130)
    HOUSE_OFFSETS = (90,145)
    HOUSE_MAP={
        'F':TOP_ROW_POSITION,
        'E':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0],TOP_ROW_POSITION[1]),
        'D':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*2,TOP_ROW_POSITION[1]),
        'C':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*3,TOP_ROW_POSITION[1]),
        'B':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*4,TOP_ROW_POSITION[1]),
        'A':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*5,TOP_ROW_POSITION[1]),
        'a':(TOP_ROW_POSITION[0],TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'b':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0],TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'c':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*2,TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'd':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*3,TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'e':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*4,TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
        'f':(TOP_ROW_POSITION[0]+HOUSE_OFFSETS[0]*5,TOP_ROW_POSITION[1]+HOUSE_OFFSETS[1]),
    }
    
    canvas = MultiCanvas(2,width=800, height=400)
    canvas[0].draw_image(Image.from_file('images/Board.png'),0,0)
    
    with hold_canvas(canvas):
        self._draw_seeds(canvas[1],HOUSE_MAP)
        self._draw_numbers(canvas[1],HOUSE_MAP)
    display(canvas)

Kalah_Game.draw_board = draw_board

### Method: _draw_seeds
The method *_draw_seeds()* reads the game of the state and accordingly draws seeds on the respective houses. Each seed gets drawn in a random position inside the house with an additional random rotation.

To place the seeds accurately we must consider the width and height of the image of one seed. To do so we subtract the width and height from the final position chosen.

In [57]:
def _draw_seeds(self,canvas,housemap):
    HOUSE_RAD = 23
    HOUSE_OFFSETS = (90,125)
    seedsprites = [Image.from_file(f'images/Samen{i}.png') for i in range(1,6)]
    seed_width = 15
    seed_height = 15
    
    gamestate = self._state_to_dict()
    for house in gamestate:
        for seed in range(gamestate[house]):
            seed_sprite = seedsprites[random.randrange(5)]
            
            if house == 'O':
                centerX, centerY = (housemap['F'][0]-HOUSE_OFFSETS[0],random.randrange(housemap['F'][1],housemap['F'][1]+HOUSE_OFFSETS[1]))
            elif house == 'o':
                centerX, centerY = (housemap['F'][0]+HOUSE_OFFSETS[0]*6,random.randrange(housemap['F'][1],housemap['F'][1]+HOUSE_OFFSETS[1]))
            else:
                centerX, centerY = housemap[house]

            r = HOUSE_RAD * math.sqrt(random.random())
            theta = random.random() * 2 * math.pi
            x = centerX + r * math.cos(theta)
            y = centerY + r * math.sin(theta)
            canvas.save()

            #Rotate by random amount for variation
            canvas.translate(x, y)
            canvas.rotate(random.uniform(0.,math.pi))
            canvas.translate(-x, -y)
            canvas.draw_image(seed_sprite, x-seed_width, y-seed_height)
            canvas.restore()

Kalah_Game._draw_seeds = _draw_seeds

### Method: _draw_numbers
The method *_draw_numbers()* reads the game of the state and accordingly draws numbers that represent the amount of seeds on the respective houses. 



In [58]:
def _draw_numbers(self,canvas,housemap):
    NUMBER_OFFSET_Y_BOT = 50
    NUMBER_OFFSET_Y_TOP = 60
    HOUSE_OFFSETS = (90,145)
    housenumbermap = {
        'F': (housemap['F'][0],housemap['F'][1]+NUMBER_OFFSET_Y_TOP),
        'E': (housemap['E'][0],housemap['E'][1]+NUMBER_OFFSET_Y_TOP),
        'D': (housemap['D'][0],housemap['D'][1]+NUMBER_OFFSET_Y_TOP),
        'C': (housemap['C'][0],housemap['C'][1]+NUMBER_OFFSET_Y_TOP),
        'B': (housemap['B'][0],housemap['B'][1]+NUMBER_OFFSET_Y_TOP),
        'A': (housemap['A'][0],housemap['A'][1]+NUMBER_OFFSET_Y_TOP),
        'a': (housemap['a'][0],housemap['a'][1]-NUMBER_OFFSET_Y_BOT),
        'b': (housemap['b'][0],housemap['b'][1]-NUMBER_OFFSET_Y_BOT),
        'c': (housemap['c'][0],housemap['c'][1]-NUMBER_OFFSET_Y_BOT),
        'd': (housemap['d'][0],housemap['d'][1]-NUMBER_OFFSET_Y_BOT),
        'e': (housemap['e'][0],housemap['e'][1]-NUMBER_OFFSET_Y_BOT),
        'f': (housemap['f'][0],housemap['f'][1]-NUMBER_OFFSET_Y_BOT),
        'O': (housemap['F'][0]-HOUSE_OFFSETS[0],housemap['F'][1]+HOUSE_OFFSETS[1]+HOUSE_OFFSETS[1]/5),
        'o': (housemap['F'][0]+HOUSE_OFFSETS[0]*6,housemap['F'][1]+HOUSE_OFFSETS[1]+HOUSE_OFFSETS[1]/5),
        
    }
    TURN_POS = (housemap['A'][0]+HOUSE_OFFSETS[0],62)

    gamestate = self._state_to_dict()
    canvas.font = '26px serif'
    canvas.text_align    = 'center'

    for key in housenumbermap:
        canvas.fill_text(gamestate[key],housenumbermap[key][0],housenumbermap[key][1])

    canvas.font = '22px serif'
    canvas.fill_text('Turn '+str(self.turn),TURN_POS[0],TURN_POS[1])
    
        
Kalah_Game._draw_numbers = _draw_numbers

### Method: _state_to_dict
The private method *_state_to_dict()* transforms a gamestate into a dictionary.

<b style="background-color:yellow; color:blue">Comments:</b>
- You should use a consistent style for indentation.
- This should be explained.

In [59]:
def _state_to_dict(self):
        d = {}
        for i in [0,1]:
            for j in range(7):
                d[gOrder[i][j]] = self.state[i][j]
        return d
Kalah_Game._state_to_dict = _state_to_dict

### Method: start
The method *start()* starts the Kalah game. Until the game is finished (the method *_finished()* returns True), both players take turns, starting with the player with the number 0. At the start of each turn, the current game state is shown. Next, the current player chooses one of the house with the <tt>Player</tt> method *choose_house(current_state)*. Afterwards, this choice is handed to the private method *_move(player_num, choice)* which calculates the new game state. The attribute *state* is updated with this new game state. Then it is the turn of the other player. If the game is finished, the method *utility()* calculates which of the players wins the game and the result is printed to the console.

In [60]:
def start(self):
    while(not finished(self.state)):
        self.turn += 1
        if self.display_mode == 1:
            print("\nCurrent state:")
            self.show_state()
            print(f"Next is {self.players[self.current_player].name}'s turn.")
        elif self.display_mode == 2:
            print("Current state:")
            self.draw_board()
            print(f"Next is {self.players[self.current_player].name}'s turn.")
            time.sleep(0.05)
        
        choice = self.players[self.current_player].choose_house(self.state)
        
        if self.display_mode != 0:
            print(f"{self.players[self.current_player].name} chose {gOrder[self.current_player][choice]}")

        self.state, another_turn, move_log = move(self.state, self.current_player, choice)
        self.logged_moves.append(move_log)
        
        if not another_turn:
            self.current_player = other_player(self.current_player)
        elif another_turn and self.display_mode != 0:
            print(f"{self.players[self.current_player].name} gets another turn!")

    won0 = utility(self.state,0)
    
    if self.display_mode != 0:
        if self.display_mode == 1:
            print("")
            self.show_state()
        elif self.display_mode == 2:
            self.draw_board()
            
        print("Finished Game!")  
        
        if(won0 == 1):
            print(f"{self.players[0]} wins!")
        elif(won0 == -1):
            print(f"{self.players[1]} wins!")
        else:
            print(f"Draw!")
    
Kalah_Game.start = start

### Method: log_to_file
The method *log_to_file* creates a json file from all important game information, including the player subclass types and names as well as the different moves from the game and the winner(s). With the use of this method, played games can be saved and analyzed. The created log file is saved to the folder "logs" with the given file_id in the file name.

In [61]:
import json

def log_to_file(self, file_id):
    json_dict = {
        "players":{
            0:{},
            1:{}
        },
        "moves":self.logged_moves,
        "winner":[i for i in range(2) if utility(self.state,i) != -1]
    }
    
    for i in range(2):
        json_dict["players"][i]["type"] = self.players[i].player_type
        json_dict["players"][i]["name"] = self.players[i].name
        if isinstance(self.players[i], Random_AI):
            json_dict["players"][i]["seed"] = self.players[i].seed
        if isinstance(self.players[i], Minimax):
            json_dict["players"][i]["limit"] = self.players[i].limit
    
    with open(f"logs/log_{file_id}.json", "w") as f:
        f.write(json.dumps(json_dict, indent=4))
        f.close()

Kalah_Game.log_to_file = log_to_file

<b style="background-color:yellow; color:blue">Comments:</b>
I would prefer if you would create plain text files instead of *Json* files.

## Tests of Kalah_Game Class

### 1. There must be exactly two players

In [62]:
try:
    game = Kalah_Game([Player(0,"Test")],0)
except ValueError as e:
    print(e)

There must be exactly two players!


### 2. Players must be instances of a Player sublass

In [63]:
try:
    game = Kalah_Game([Player(0,"Test"), Player(1,"Test")],0)
except ValueError as e:
    print(e)

Both players must be of instances of a subclass of the class Player!


### 3. Player numbers must be 0 and 1

In [64]:
try:
    game = Kalah_Game([Human(1,"Human"), Random_AI(1,"AI",2)],0)
except ValueError as e:
    print(e)

One of the players must be self.number 0 and the other one self.number 1!


### 4. Display mode must be 0, 1 or 2

In [65]:
try:
    game = Kalah_Game([Human(1,"Human"), Random_AI(0,"AI",2)],3)
except ValueError as e:
    print(e)

The display mode must be 0, 1 or 2!


### 5. Successful Game creation with two AIs

In [66]:
game = Kalah_Game([Random_AI(0,"Rando",1), Minimax(1,"Mini",3)], 2)

In [67]:
%%time
game.start()

Current state:


MultiCanvas(height=400, width=800)

Next is Rando's turn.
Rando chose F
Current state:


MultiCanvas(height=400, width=800)

Next is Mini's turn.
Mini chose b
Mini gets another turn!
Current state:


...

Next is Mini's turn.
Mini chose d
Current state:


MultiCanvas(height=400, width=800)

Next is Rando's turn.
Rando chose C


MultiCanvas(height=400, width=800)

Finished Game!
Mini wins!
Wall time: 2.78 s


In [68]:
timestamp = str(dt.datetime.now()).replace(':','.')
game.log_to_file(timestamp)

## Repeat Game from Log File

The function *repeat_game* takes the filename of a log file and the display mode with which the game should be repeated as the input values. It also receives the information, if included AIs should calculate their choices new or if they should just repeat the moves that are presented in the log file. The decicions of human players are always repeated based on the log file. 

The function detects the player types and names from the file and creates players based on this information. The player types are only relevant for AIs if **repeat_AIs** is set to False. In all other cases, the players are created from the <tt>Repeated_Player</tt> class.

In the end, the game is initialized with the created players and the given display mode and then started.

In [69]:
def repeat_game(filename, repeat_AIs, display_mode):
    json_dict = {}
    
    with open("logs/"+filename) as f:
        json_dict = json.load(f)
    
    players = []
    for i in range(2):
        i = str(i)
        p_type = json_dict["players"][i]["type"]
        
        if repeat_AIs:
            players.append(Repeated_Player(int(i),json_dict["players"][i]["name"],json_dict["moves"]))
        else:
            if p_type == "Human":
                players.append(Repeated_Player(int(i),json_dict["players"][i]["name"],json_dict["moves"]))
            elif p_type == "Random_AI":
                players.append(Random_AI(int(i),json_dict["players"][i]["name"],json_dict["players"][i]["seed"]))
            elif p_type == "Minimax":
                players.append(Minimax(int(i),json_dict["players"][i]["name"],json_dict["players"][i]["limit"]))
            else:
                raise ValueError("The given player type is unknown!")
    
    game = Kalah_Game([players[0],players[1]],display_mode)
    game.start()

### Tests of Repeat Game
#### Repeating the game of two AIs, where the decisions are repeated based on the log file (Player types: Repeated_Player)

In [70]:
%%time
repeat_game(f"log_{timestamp}.json",True,1)


Current state:
Rando:			O: 0  F: 4  E: 4  D: 4  C: 4  B: 4  A: 4  
Mini:			a: 4  b: 4  c: 4  d: 4  e: 4  f: 4  o: 0  

Next is Rando's turn.
Rando chose F

Current state:
Rando:			O: 1  F: 0  E: 4  D: 4  C: 4  B: 4  A: 4  
Mini:			a: 5  b: 5  c: 5  d: 4  e: 4  f: 4  o: 0  

Next is Mini's turn.
Mini chose b
Mini gets another turn!

Current state:
Rando:			O: 1  F: 0  E: 4  D: 4  C: 4  B: 4  A: 4  
Mini:			a: 5  b: 0  c: 6  d: 5  e: 5  f: 5  o: 1  

Next is Mini's turn.
Mini chose c

Current state:
Rando:			O: 1  F: 0  E: 4  D: 4  C: 4  B: 5  A: 5  
Mini:			a: 5  b: 0  c: 0  d: 6  e: 6  f: 6  o: 2  

Next is Rando's turn.
Rando chose A

Current state:
Rando:			O: 7  F: 0  E: 5  D: 5  C: 5  B: 6  A: 0  
Mini:			a: 0  b: 0  c: 0  d: 6  e: 6  f: 6  o: 2  

Next is Mini's turn.
Mini chose d

Current state:
Rando:			O: 7  F: 0  E: 5  D: 5  C: 6  B: 7  A: 1  
Mini:			a: 0  b: 0  c: 0  d: 0  e: 7  f: 7  o: 3  

Next is Rando's turn.
Rando chose A

Current state:
Rando:			O: 7  F: 0  E: 5  D: 

#### Repeating the game of two AIs, where both of them calculate their decisions new (Player types: Random_AI)

In [71]:
%%time
repeat_game(f"log_{timestamp}.json",False,1)


Current state:
Rando:			O: 0  F: 4  E: 4  D: 4  C: 4  B: 4  A: 4  
Mini:			a: 4  b: 4  c: 4  d: 4  e: 4  f: 4  o: 0  

Next is Rando's turn.
Rando chose B

Current state:
Rando:			O: 0  F: 5  E: 5  D: 5  C: 5  B: 0  A: 4  
Mini:			a: 4  b: 4  c: 4  d: 4  e: 4  f: 4  o: 0  

Next is Mini's turn.
Mini chose c
Mini gets another turn!

Current state:
Rando:			O: 0  F: 5  E: 5  D: 5  C: 5  B: 0  A: 4  
Mini:			a: 4  b: 4  c: 0  d: 5  e: 5  f: 5  o: 1  

Next is Mini's turn.
Mini chose d

Current state:
Rando:			O: 0  F: 5  E: 5  D: 5  C: 5  B: 1  A: 5  
Mini:			a: 4  b: 4  c: 0  d: 0  e: 6  f: 6  o: 2  

Next is Rando's turn.
Rando chose E

Current state:
Rando:			O: 1  F: 6  E: 0  D: 5  C: 5  B: 1  A: 5  
Mini:			a: 5  b: 5  c: 1  d: 0  e: 6  f: 6  o: 2  

Next is Mini's turn.
Mini chose c

Current state:
Rando:			O: 1  F: 6  E: 0  D: 5  C: 0  B: 1  A: 5  
Mini:			a: 5  b: 5  c: 0  d: 0  e: 6  f: 6  o: 8  

Next is Rando's turn.
Rando chose A

Current state:
Rando:			O: 1  F: 7  E: 1  D: 

## Test: Random_AI vs. Minimax

<b style="background-color:yellow; color:blue">Comments:</b> I need an explanation what is going on here.  And the whole thing should be turned into a function.

In [ ]:
%%time

limit = 2

mini_wins = 0
rando_wins = 0

for seed in range(100):
    game1 = Kalah_Game([Random_AI(0,"Rando",seed), Minimax(1,"Mini",limit)],0)
    game1.start()
    won1 = utility(game1.state, 0)
    
    # print("")
    # game1.show_state() 
    if(won1 == 1):
        # print(f"{game1.players[0]} wins!")
        rando_wins += 1
    elif(won1 == -1):
        # print(f"{game1.players[1]} wins!")
        mini_wins += 1
    else:
        # print(f"Draw!")
        pass
        
    ############################
    
    game2 = Kalah_Game([Minimax(0,"Mini",limit), Random_AI(1,"Rando",seed)],0)
    game2.start()
    won2 = utility(game2.state, 0)
    
    # print("")
    # game2.show_state() 
    if(won2 == 1):
        # print(f"{game2.players[0]} wins!")
        mini_wins += 1
    elif(won2 == -1):
        # print(f"{game2.players[1]} wins!")
        rando_wins += 1
    else:
        # print(f"Draw!")
        pass
            
print(f"Rando won {rando_wins} times and Mini won {mini_wins} times!")

## Test: Minimax vs. Minimax with seeds 2 and 3

In [72]:
game = Kalah_Game([Minimax(0,"Mini_3",3),Minimax(1,"Mini_2",2)],0)
game.start()
won = utility(game.state, 0)

if(won == 1):
    print(f"{game.players[0]} wins!")
elif(won == -1):
    print(f"{game.players[1]} wins!")
else:
    print(f"Draw!")

Mini_3 wins!


## Play against Minimax

In [73]:
gameHuman = Kalah_Game([Human(0,"Karl"), Minimax(1,"AI2",4)], 2)

In [74]:
gameHuman.start()

Karl chose E


MultiCanvas(height=400, width=800)

Finished Game!
Karl wins!


## <b style="background-color:yellow; color:blue">Concluding Remarks</b> 
* I really like the graphical representation of the game.
  However, this should be separated into a different notebook so that I can reuse it for my class.
* The work should be structured into serveral notebooks.  I do not want to execute all tests when I just want to play a game.
  I usually just press the button `Run All Above` and that runs the tests.
* Testing should be more systematic, i.e. you should structure the code into functions and you should also discuss the results.
* You still need to implement *Alpha-Beta-Pruning* and *[Scout](https://www.aaai.org/Papers/AAAI/1980/AAAI80-041.pdf)*. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b91c3ea7-d814-439b-837a-72fdc90697b1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>